### Data extraction and feature extraction（new version)

In [53]:
import pretty_midi
import pandas as pd
from pathlib import Path
from collections import defaultdict

In [54]:
import zipfile
import io
from collections import defaultdict

In [91]:
def extract_piano_note_features(midi_file, filename=None, composer=None):
    """Extract features from a MIDI file - handles both file paths and binary data"""
    try:
        # Handle both file paths (strings) and binary data
        if isinstance(midi_file, str):
            pm = pretty_midi.PrettyMIDI(midi_file)
        else:
            # For binary data from zip files
            with io.BytesIO(midi_file) as buf:
                pm = pretty_midi.PrettyMIDI(buf)
        
        # Find the first piano instrument if available
        instrument = None
        for inst in pm.instruments:
            instrument = inst
            break  # Just take the first instrument for now
        
        if instrument is None:
            return pd.DataFrame()
            
        notes = defaultdict(list)
        for note in instrument.notes:
            notes["pitch"].append(note.pitch)
            notes["velocity"].append(note.velocity)
            notes["note_name"].append(pretty_midi.note_number_to_name(note.pitch))
            notes["octave"].append(note.pitch // 12 - 1)
            notes["start"].append(note.start)
            notes["end"].append(note.end)
            notes["duration"].append(note.end - note.start)
            notes["instrument_name"].append(instrument.program)  # Store program number instead of name
            
        # Add filename and composer if provided
        df = pd.DataFrame(notes)
        if filename:
            df["filename"] = filename
        if composer:
            df["composer"] = composer
            
        return df
        
    except Exception as e:
        print(f"Failed to process {filename if filename else midi_file}: {e}")
        return pd.DataFrame()  # Return empty if failed

def extract_all_piano_midi_files_from_zips(folder):
    # 查找所有ZIP文件
    zip_paths = list(Path(folder).glob("*.zip"))
    print(f"Found {len(zip_paths)} ZIP files")
    
    all_dfs = []
    midi_count = 0
    processed_files = 0
    
    for zip_path in zip_paths:
        # 从ZIP文件名提取作曲家名称
        composer = zip_path.stem  # 使用不带扩展名的文件名作为作曲家名称
        print(f"Processing ZIP: {zip_path} - Composer: {composer}")
        
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                # 列出ZIP文件中的所有文件
                midi_files = [f for f in zip_ref.namelist() if f.lower().endswith('.mid')]
                print(f"  Found {len(midi_files)} MIDI files in {zip_path.name}")
                
                for midi_file in midi_files:
                    # 读取MIDI文件数据
                    midi_data = zip_ref.read(midi_file)
                    
                    # 创建有意义的文件名（ZIP名称 + MIDI文件名）
                    full_filename = f"{composer}/{midi_file}"
                    
                    # 处理MIDI数据，并传递作曲家信息
                    df = extract_piano_note_features(midi_data, full_filename, composer)
                    processed_files += 1
                    
                    if not df.empty:
                        all_dfs.append(df)
                        midi_count += 1
                    
                    if processed_files % 10 == 0:
                        print(f"  Processed {processed_files} files, successfully extracted data from {midi_count} files")
        except Exception as e:
            print(f"Error processing ZIP {zip_path}: {e}")
    
    print(f"Total processed: {processed_files} files")
    print(f"Successfully extracted data from {midi_count} MIDI files across {len(zip_paths)} ZIP archives")
    
    if all_dfs:
        return pd.concat(all_dfs, ignore_index=True)
    else:
        return pd.DataFrame()
# 使用方法
# data = extract_all_piano_midi_files_from_zips("piano_folder")

In [92]:
def enrich_midi_features(df):
    """Add musical context features to an existing note dataframe"""
    if df.empty:
        return df
    
    # Create a copy to avoid modifying the original
    enriched_df = df.copy()
    
    # Add pitch class (normalize octave information)
    enriched_df['pitch_class'] = enriched_df['pitch'] % 12
    
    # Sort by filename and start time
    enriched_df = enriched_df.sort_values(['filename', 'start'])
    
    # Group by filename to process each piece separately
    processed_dfs = []
    
    for filename, piece_df in enriched_df.groupby('filename'):
        # Calculate intervals between consecutive notes
        piece_df['next_interval'] = piece_df['pitch'].diff().shift(-1).fillna(0).astype(int)
        
        # Identify chords - notes that start at the same time
        piece_df['time_group'] = (piece_df['start'] * 100).round() / 100  # Group within 10ms
        piece_df['chord_size'] = piece_df.groupby('time_group')['time_group'].transform('count')
        piece_df['is_chord_note'] = piece_df['chord_size'] > 1
        
        # Add metrical position features
        # Create time windows (e.g., 0.5 second windows)
        piece_df['beat_window'] = (piece_df['start'] // 0.5)
        
        # Flag downbeats (strong beat positions)
        # Simple approximation: notes at the beginning of each window
        piece_df['is_downbeat'] = False
        downbeat_idxs = piece_df.groupby('beat_window')['start'].idxmin()
        piece_df.loc[downbeat_idxs, 'is_downbeat'] = True
        
        # Note density in local context (notes per beat window)
        window_counts = piece_df.groupby('beat_window').size()
        piece_df['local_density'] = piece_df['beat_window'].map(window_counts)
        
        # Clean up temporary columns
        piece_df = piece_df.drop(['time_group', 'beat_window'], axis=1)
        
        processed_dfs.append(piece_df)
    
    # Combine all processed pieces
    result_df = pd.concat(processed_dfs, ignore_index=True)
    
    return result_df

In [93]:
# read data and extract pitch, velocity, note_name, octave, start_time, end, and duration 
# data_dir = "/Users/yang/Desktop/Yale Spring 2025/CPSC 552 Deep learning theory and applications /DeepL project - music generation /Data set/lmd_matched"
all_notes_df = extract_all_piano_midi_files_from_zips("piano")
# Apply the enrichment to your existing dataframe
enriched_notes_df = enrich_midi_features(all_notes_df)

Found 19 ZIP files
Processing ZIP: piano/bach.zip - Composer: bach
  Found 3 MIDI files in bach.zip
Processing ZIP: piano/tschai.zip - Composer: tschai
  Found 12 MIDI files in tschai.zip
  Processed 10 files, successfully extracted data from 10 files
Processing ZIP: piano/schubert.zip - Composer: schubert
  Found 29 MIDI files in schubert.zip
  Processed 20 files, successfully extracted data from 20 files
  Processed 30 files, successfully extracted data from 30 files
  Processed 40 files, successfully extracted data from 40 files
Processing ZIP: piano/mendelssohn.zip - Composer: mendelssohn
  Found 15 MIDI files in mendelssohn.zip
  Processed 50 files, successfully extracted data from 50 files
Processing ZIP: piano/balakir.zip - Composer: balakir
  Found 1 MIDI files in balakir.zip
  Processed 60 files, successfully extracted data from 60 files
Processing ZIP: piano/debussy.zip - Composer: debussy
  Found 9 MIDI files in debussy.zip
Processing ZIP: piano/muss.zip - Composer: muss
  F

In [94]:
display(all_notes_df)

,pitch,velocity,note_name,octave,start,end,duration,instrument_name,filename,composer
0,72,96,C5,5,0.003061,0.139232,0.136171,0,bach/bach_847.mid,bach
1,63,80,D#4,4,0.118555,0.250134,0.131579,0,bach/bach_847.mid,bach
2,62,77,D4,4,0.230397,0.363021,0.132623,0,bach/bach_847.mid,bach
3,63,77,D#4,4,0.343179,0.481835,0.138655,0,bach/bach_847.mid,bach
4,60,80,C4,4,0.465168,0.601572,0.136404,0,bach/bach_847.mid,bach
...,...,...,...,...,...,...,...,...,...,...
381243,76,67,E5,5,109.332153,109.630661,0.298507,0,schumann/scn68_12.mid,schumann
381244,71,67,B4,4,109.332153,109.630661,0.298507,0,schumann/scn68_12.mid,schumann
381245,81,92,A5,5,109.630661,110.112589,0.481928,0,schumann/scn68_12.mid,schumann
381246,76,77,E5,5,109.630661,110.112589,0.481928,0,schumann/scn68_12.mid,schumann


In [61]:
enriched_notes_df = enrich_midi_features(all_notes_df)
display(enriched_notes_df)

,pitch,velocity,note_name,octave,start,end,duration,instrument_name,filename,composer,pitch_class,next_interval,chord_size,is_chord_note,is_downbeat,local_density
0,81,60,A5,5,0.321096,0.642192,0.321096,0,albeniz/alb_esp1.mid,albeniz,9,7,1,False,True,1
1,88,66,E6,6,0.642192,2.574745,1.932553,0,albeniz/alb_esp1.mid,albeniz,4,-2,1,False,True,1
2,86,55,D6,6,2.574745,2.641412,0.066667,0,albeniz/alb_esp1.mid,albeniz,2,2,1,False,True,4
3,88,47,E6,6,2.641412,2.708079,0.066667,0,albeniz/alb_esp1.mid,albeniz,4,-2,1,False,False,4
4,86,62,D6,6,2.708079,2.971262,0.263184,0,albeniz/alb_esp1.mid,albeniz,2,-2,1,False,False,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381243,71,71,B4,4,159.353994,159.635684,0.281690,0,tschai/ty_september.mid,tschai,11,3,4,True,False,4
381244,74,71,D5,5,159.353994,159.635684,0.281690,0,tschai/ty_september.mid,tschai,2,-7,4,True,False,4
381245,67,88,G4,4,159.917374,161.408058,1.490684,0,tschai/ty_september.mid,tschai,7,-5,3,True,True,3
381246,62,79,D4,4,159.917374,161.408058,1.490684,0,tschai/ty_september.mid,tschai,2,-3,3,True,False,3


In [95]:
enriched_notes_df["filename"].nunique() # check number of unique files in our code 

295

### Data preparation & Preprocessing 


In [109]:
# Full pipeline for symbolic MusicGen-style training using extracted note features (with chord conditioning)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from collections import Counter

# Configuration
D_MODEL = 256
NUM_PITCHES = 128
NUM_VELOCITIES = 32
NUM_DURATIONS = 32
NUM_CHORDS = 12  # 12 pitch classes (C, C#, D, ..., B)
NUM_INSTRUMENTS = 12
SEQ_LEN = 64



NUM_PITCH_CLASSES = 12
NUM_INTERVALS = 24          # 根据你的数据最大间隔
MAX_CHORD_SIZE = 10         # 根据你的数据中最大和弦音数
NUM_CHORD_SIZES = MAX_CHORD_SIZE + 1  # 如果从0开始
NUM_DENSITY_BUCKETS = 10    # 取决于你local_density怎么分箱



In [110]:
# Chord Estimation from Notes (chromagram-inspired)
def estimate_chords(df):
    df = df.copy()
    df["chord"] = -1
    filenames = df["filename"].unique()
    for fname in filenames:
        song = df[df["filename"] == fname].copy()
        song = song.sort_values("start")
        chords = []
        for i in range(0, len(song), SEQ_LEN):
            segment = song.iloc[i:i+SEQ_LEN]
            pitch_classes = [p % 12 for p in segment["pitch"]]
            if len(pitch_classes) == 0:
                chord_id = 0
            else:
                chord_id = Counter(pitch_classes).most_common(1)[0][0]
            chords += [chord_id] * len(segment)
        df.loc[df["filename"] == fname, "chord"] = chords
    return df

In [111]:
def discretize_velocity(velocity):
    return min(int(velocity // 4), NUM_VELOCITIES - 1)

def discretize_duration(duration):
    idx = np.floor(duration / 0.1).astype(int)
    return np.clip(idx, 0, NUM_DURATIONS - 1)

def discretize_next_interval(interval):
    return np.clip(interval, 0, NUM_INTERVALS - 1)

def discretize_chord_size(size):
    return np.clip(size, 0, NUM_CHORD_SIZES - 1)

def discretize_local_density(density):
    # 如果 local_density 是连续值但想做 embedding → 需要分箱
    idx = np.floor(density / 0.1).astype(int)  # 举例，每0.1一个bin
    return np.clip(idx, 0, NUM_DENSITY_BUCKETS - 1)

def build_sequence_tensor(df, max_seq_len=SEQ_LEN):
    sequences = []
    grouped = df.groupby("filename")
    
    unique_instruments = df["instrument_name"].unique()
    instrument_to_idx = {name: idx for idx, name in enumerate(unique_instruments)}
    
    all_durations = []
    all_velocities = []
    all_pitch_classes = []
    all_next_intervals = []
    all_chord_sizes = []
    all_local_density = []

    for _, group in grouped:
        group = group.sort_values("start")
        for i in range(0, len(group) - max_seq_len, max_seq_len):
            chunk = group.iloc[i:i+max_seq_len]
            
            pitch = torch.tensor(chunk["pitch"].values, dtype=torch.long)
            velocity = torch.tensor(chunk["velocity"].apply(discretize_velocity).values, dtype=torch.long)
            duration = torch.tensor(chunk["duration"].apply(discretize_duration).values, dtype=torch.long)
            chord = torch.tensor(chunk["chord"].values, dtype=torch.long)
            
            instrument_indices = [instrument_to_idx[name] for name in chunk["instrument_name"].values]
            instrument = torch.tensor(instrument_indices, dtype=torch.long)
            
            pitch_class = torch.tensor(chunk["pitch_class"].values, dtype=torch.long)
            next_interval = torch.tensor(chunk["next_interval"].apply(discretize_next_interval).values, dtype=torch.long)
            chord_size = torch.tensor(chunk["chord_size"].apply(discretize_chord_size).values, dtype=torch.long)
            is_chord_note = torch.tensor(chunk["is_chord_note"].values, dtype=torch.long)
            is_downbeat = torch.tensor(chunk["is_downbeat"].values, dtype=torch.long)
            
            # local_density: 是否离散化取决于模型 → 如果 embedding，需要离散化
            local_density = torch.tensor(chunk["local_density"].apply(discretize_local_density).values, dtype=torch.long)
            # 如果 continuous → local_density = torch.tensor(chunk["local_density"].values, dtype=torch.float)

            sequences.append((
                pitch, velocity, duration, chord, instrument,
                pitch_class, next_interval, chord_size, is_chord_note, is_downbeat, local_density
            ))

            # Debug: 记录max/min
            all_durations.extend(duration.tolist())
            all_velocities.extend(velocity.tolist())
            all_pitch_classes.extend(pitch_class.tolist())
            all_next_intervals.extend(next_interval.tolist())
            all_chord_sizes.extend(chord_size.tolist())
            all_local_density.extend(local_density.tolist())

    # 打印所有离散化后的值范围
    print("Duration max:", max(all_durations), "min:", min(all_durations))
    print("Velocity max:", max(all_velocities), "min:", min(all_velocities))
    print("Pitch class max:", max(all_pitch_classes), "min:", min(all_pitch_classes))
    print("Next interval max:", max(all_next_intervals), "min:", min(all_next_intervals))
    print("Chord size max:", max(all_chord_sizes), "min:", min(all_chord_sizes))
    print("Local density max:", max(all_local_density), "min:", min(all_local_density))
    
    return sequences


In [112]:
# Dataset Wrapper
class SequenceDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx]

In [113]:
# df = estimate_chords(all_notes_df)
df = estimate_chords(enriched_notes_df)
sequences = build_sequence_tensor(df)
dataset = SequenceDataset(sequences)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

Duration max: 31 min: 0
Velocity max: 30 min: 1
Pitch class max: 11 min: 0
Next interval max: 23 min: 0
Chord size max: 7 min: 1
Local density max: 9 min: 9


In [114]:
display(df)

,pitch,velocity,note_name,octave,start,end,duration,instrument_name,filename,composer,pitch_class,next_interval,chord_size,is_chord_note,is_downbeat,local_density,chord
0,81,60,A5,5,0.321096,0.642192,0.321096,0,albeniz/alb_esp1.mid,albeniz,9,7,1,False,True,1,4
1,88,66,E6,6,0.642192,2.574745,1.932553,0,albeniz/alb_esp1.mid,albeniz,4,-2,1,False,True,1,4
2,86,55,D6,6,2.574745,2.641412,0.066667,0,albeniz/alb_esp1.mid,albeniz,2,2,1,False,True,4,4
3,88,47,E6,6,2.641412,2.708079,0.066667,0,albeniz/alb_esp1.mid,albeniz,4,-2,1,False,False,4,4
4,86,62,D6,6,2.708079,2.971262,0.263184,0,albeniz/alb_esp1.mid,albeniz,2,-2,1,False,False,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381243,71,71,B4,4,159.353994,159.635684,0.281690,0,tschai/ty_september.mid,tschai,11,3,4,True,False,4,7
381244,74,71,D5,5,159.353994,159.635684,0.281690,0,tschai/ty_september.mid,tschai,2,-7,4,True,False,4,7
381245,67,88,G4,4,159.917374,161.408058,1.490684,0,tschai/ty_september.mid,tschai,7,-5,3,True,True,3,7
381246,62,79,D4,4,159.917374,161.408058,1.490684,0,tschai/ty_september.mid,tschai,2,-3,3,True,False,3,7


### Model construction 

In [115]:
class SymbolicMusicTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        self.pitch_embed = nn.Embedding(NUM_PITCHES, D_MODEL)
        self.velocity_embed = nn.Embedding(NUM_VELOCITIES, D_MODEL)
        self.duration_embed = nn.Embedding(NUM_DURATIONS, D_MODEL)
        self.chord_embed = nn.Embedding(NUM_CHORDS, D_MODEL)
        self.instrument_embed = nn.Embedding(NUM_INSTRUMENTS, D_MODEL)
        
        self.pitch_class_embed = nn.Embedding(NUM_PITCH_CLASSES, D_MODEL)
        self.next_interval_embed = nn.Embedding(NUM_INTERVALS, D_MODEL)
        self.chord_size_embed = nn.Embedding(NUM_CHORD_SIZES, D_MODEL)
        self.is_chord_note_embed = nn.Embedding(2, D_MODEL)
        self.is_downbeat_embed = nn.Embedding(2, D_MODEL)
        self.local_density_proj = nn.Linear(1, D_MODEL)  # assuming local_density is numeric

        self.pos_embed = nn.Embedding(SEQ_LEN, D_MODEL)

        encoder_layer = nn.TransformerEncoderLayer(d_model=D_MODEL, nhead=8, dim_feedforward=512)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=4)

        self.pitch_out = nn.Linear(D_MODEL, NUM_PITCHES)
        self.velocity_out = nn.Linear(D_MODEL, NUM_VELOCITIES)
        self.duration_out = nn.Linear(D_MODEL, NUM_DURATIONS)
        self.instrument_out = nn.Linear(D_MODEL, NUM_INSTRUMENTS)

    def forward(self, pitch, velocity, duration, chord, instrument,
                pitch_class, next_interval, chord_size, is_chord_note, is_downbeat, local_density):
        B, T = pitch.shape
        pos = torch.arange(T, device=pitch.device).unsqueeze(0).expand(B, T)

        x = self.pitch_embed(pitch) + \
            self.velocity_embed(velocity) + \
            self.duration_embed(duration) + \
            self.chord_embed(chord) + \
            self.instrument_embed(instrument) + \
            self.pitch_class_embed(pitch_class) + \
            self.next_interval_embed(next_interval) + \
            self.chord_size_embed(chord_size) + \
            self.is_chord_note_embed(is_chord_note) + \
            self.is_downbeat_embed(is_downbeat) + \
            self.local_density_proj(local_density.unsqueeze(-1)) + \
            self.pos_embed(pos)

        x = x.transpose(0, 1)
        x = self.transformer(x)
        x = x.transpose(0, 1)

        return self.pitch_out(x), self.velocity_out(x), self.duration_out(x), self.instrument_out(x)


In [120]:
def train_model(model, dataloader, epochs=7):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        for pitch, velocity, duration, chord, instrument, pitch_class, next_interval, chord_size, is_chord_note, is_downbeat, local_density in dataloader:
            optimizer.zero_grad()
            local_density = local_density.float()
            pitch_logits, vel_logits, dur_logits, instr_logits = model(
                pitch, velocity, duration, chord, instrument,
                pitch_class, next_interval, chord_size, is_chord_note, is_downbeat, local_density
            )
            
            loss = F.cross_entropy(pitch_logits.view(-1, NUM_PITCHES), pitch.view(-1)) + \
                   F.cross_entropy(vel_logits.view(-1, NUM_VELOCITIES), velocity.view(-1)) + \
                   F.cross_entropy(dur_logits.view(-1, NUM_DURATIONS), duration.view(-1)) + \
                   F.cross_entropy(instr_logits.view(-1, NUM_INSTRUMENTS), instrument.view(-1))
        
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.4f}")


In [121]:
model = SymbolicMusicTransformer()
train_model(model, dataloader)

Epoch 1, Loss: 4.3373
Epoch 2, Loss: 0.1237
Epoch 3, Loss: 0.0336
Epoch 4, Loss: 0.0153
Epoch 5, Loss: 0.0088
Epoch 6, Loss: 0.0057
Epoch 7, Loss: 0.0040


In [122]:
import torch
import pretty_midi

def generate_music_with_chord_sequence_mul(model, chord_list, output_path="generated_multi_chord_sample.mid"):
    model.eval()
    with torch.no_grad():
        T = SEQ_LEN

        # Build the chord sequence
        frames_per_chord = T // len(chord_list)
        chord_sequence = []
        for chord in chord_list:
            chord_sequence += [chord] * frames_per_chord
        if len(chord_sequence) < T:
            chord_sequence += [chord_list[-1]] * (T - len(chord_sequence))
        chord_input = torch.tensor(chord_sequence, dtype=torch.long).unsqueeze(0)  # [1, T]

        # Initialize zero/fixed inputs for other features
        pitch_input = torch.zeros((1, T), dtype=torch.long)
        velocity_input = torch.zeros((1, T), dtype=torch.long)
        duration_input = torch.zeros((1, T), dtype=torch.long)
        instrument_input = torch.zeros((1, T), dtype=torch.long)

        # NEW features (initialize as zeros or fixed defaults)
        pitch_class_input = torch.zeros((1, T), dtype=torch.long)        # 0 = C
        next_interval_input = torch.zeros((1, T), dtype=torch.long)      # 0 = no interval
        chord_size_input = torch.ones((1, T), dtype=torch.long) * 3      # default triad
        is_chord_note_input = torch.zeros((1, T), dtype=torch.long)      # assume not chord note
        is_downbeat_input = torch.zeros((1, T), dtype=torch.long)        # assume not downbeat

        # local_density: 注意如果模型用 projection → float；如果 embedding → long
        local_density_input = torch.zeros((1, T), dtype=torch.float)     # for nn.Linear
        # 如果你用 embedding → local_density_input = torch.zeros((1, T), dtype=torch.long)

        # Forward pass
        pitch_logits, velocity_logits, duration_logits, instrument_logits = model(
            pitch_input, velocity_input, duration_input, chord_input, instrument_input,
            pitch_class_input, next_interval_input, chord_size_input, is_chord_note_input, is_downbeat_input, local_density_input
        )

        # Argmax to decode
        generated_pitch = pitch_logits.argmax(-1).squeeze(0).tolist()
        generated_velocity = velocity_logits.argmax(-1).squeeze(0).tolist()
        generated_duration = duration_logits.argmax(-1).squeeze(0).tolist()
        generated_instrument = instrument_logits.argmax(-1).squeeze(0).tolist()

        # Save to MIDI
        save_generated_midi(
            generated_pitch,
            generated_velocity,
            generated_duration,
            generated_instrument,
            output_path=output_path
        )


In [123]:
def save_generated_midi(pitch_seq, velocity_seq, duration_seq, instrument_seq, output_path="generated_output.mid"):
    midi = pretty_midi.PrettyMIDI()
    instrument_map = {}

    time = 0.0
    for p, v, d, i in zip(pitch_seq, velocity_seq, duration_seq, instrument_seq):
        dur_sec = (d + 1) * 0.1  # duration bucket to seconds
        velocity_clipped = min(max(int(v * 4 + 30), 1), 127)

        start_time = time
        end_time = time + dur_sec

        if i not in instrument_map:
            program_num = int(i) if i < 127 else 127
            instrument_map[i] = pretty_midi.Instrument(program=program_num)

        note = pretty_midi.Note(velocity=velocity_clipped, pitch=p, start=start_time, end=end_time)
        instrument_map[i].notes.append(note)
        time += dur_sec

    for inst in instrument_map.values():
        midi.instruments.append(inst)

    midi.write(output_path)
    print(f"MIDI file saved to: {output_path}")

In [128]:
# Example Usage
# Assuming model is already trained or initialized
chord_list = [5,3,6,7]
output_path = "generated_multi_chord_sample_new.mid"
generate_music_with_chord_sequence_mul(model, chord_list, output_path)

MIDI file saved to: generated_multi_chord_sample_new.mid
